In [1]:
# Importar librerías necesarias
import cv2
import numpy as np
import pandas as pd
import mediapipe as mp
import os
import datetime
import json

In [2]:
video_dir = 'C:\\Tesis\\TestErgo\\muestra'

In [3]:
def load_videos(video_dir):
    videos = []
    for filename in os.listdir(video_dir):
        if filename.endswith(".mp4"):
            videos.append(os.path.join(video_dir, filename))
    return videos

In [4]:
# Cargar videos
videos = load_videos(video_dir)

In [5]:
# Función para extraer frames de los videos
def extract_frames(video_path, interval=30):
    cap = cv2.VideoCapture(video_path)
    frames = []
    count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if count % interval == 0:
            frames.append(frame)
        count += 1
    cap.release()
    return frames


In [6]:
# Obtener y mostrar los nombres de todos los videos
for video_path in videos:
    video_name = os.path.basename(video_path)
    print(video_name)

etiquetado.mp4


In [7]:
# Inicializar MediaPipe para detección de puntos clave
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Función para calcular el ángulo entre tres puntos
def calculate_angle(a, b, c, d=None):
    if d is None:
        # Calcula el ángulo entre a, b y c
        a = np.array(a)  # Primer punto
        b = np.array(b)  # Segundo punto (vértice)
        c = np.array(c)  # Tercer punto

        radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
        angle = np.abs(radians * 180.0 / np.pi)

        if angle > 180.0:
            angle = 360.0 - angle

        return angle
    else:
        # Calcula el ángulo entre a, b, c y d
        a = np.array(a)  # Primer punto
        b = np.array(b)  # Segundo punto (vértice)
        c = np.array(c)  # Tercer punto
        d = np.array(d)  # Cuarto punto

        radians = np.arctan2(d[1] - c[1], d[0] - c[0]) - np.arctan2(b[1] - a[1], b[0] - a[0])
        angle = np.abs(radians * 180.0 / np.pi)

        if angle > 180.0:
            angle = 360.0 - angle

        return angle


In [8]:
# Función para detectar puntos clave usando MediaPipe
def detect_keypoints(video_path):
    cap = cv2.VideoCapture(video_path)
    pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)
    keypoints_list = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        if results.pose_landmarks:
            keypoints = [(lm.x, lm.y, lm.z) for lm in results.pose_landmarks.landmark]
            keypoints_list.append(keypoints)
        else:
            keypoints_list.append([])  # Agregar una lista vacía si no se detectan keypoints

    cap.release()
    pose.close()
    return keypoints_list

In [9]:
# Función para analizar videos masivos y guardar resultados en un solo JSON
'''def analyze_videos_in_folder(video_folder, output_json_path):
    all_analysis_results = []
    
    for video_file in os.listdir(video_folder):
        if video_file.endswith(('.mp4', '.avi', '.mov')):
            video_path = os.path.join(video_folder, video_file)
            video_name = os.path.splitext(video_file)[0]
            
            keypoints_list = detect_keypoints(video_path)
            cap = cv2.VideoCapture(video_path)
            fps = int(cap.get(cv2.CAP_PROP_FPS))
            cap.release()

            analysis_results = analyze_keypoints(keypoints_list, video_name, fps)
            all_analysis_results.extend(analysis_results)
            draw_keypoints_and_angles(video_path, analysis_results, output_folder, output_video_folder, json_folder)
    
    combined_json_path = os.path.join(json_folder, 'Resultado_analisis_combinados.json')
    with open(output_json_path, 'w') as json_file:
        json.dump(all_analysis_results, json_file, indent=4)

    print(f"Resultados del análisis exportados a: {output_json_path}")

# Ejemplo de uso:
#video_folder = 'C:\\Tesis\\TestErgo\\muestra'
video_folder = video_dir #'C:\\Tesis\\TestErgo\\VIDEOS MAQUILAS'
output_json_path = 'C:\\Tesis\\TestErgo\\analisis_videos.json'

analyze_videos_in_folder(video_folder, output_json_path)'''


'def analyze_videos_in_folder(video_folder, output_json_path):\n    all_analysis_results = []\n    \n    for video_file in os.listdir(video_folder):\n        if video_file.endswith((\'.mp4\', \'.avi\', \'.mov\')):\n            video_path = os.path.join(video_folder, video_file)\n            video_name = os.path.splitext(video_file)[0]\n            \n            keypoints_list = detect_keypoints(video_path)\n            cap = cv2.VideoCapture(video_path)\n            fps = int(cap.get(cv2.CAP_PROP_FPS))\n            cap.release()\n\n            analysis_results = analyze_keypoints(keypoints_list, video_name, fps)\n            all_analysis_results.extend(analysis_results)\n            draw_keypoints_and_angles(video_path, analysis_results, output_folder, output_video_folder, json_folder)\n    \n    combined_json_path = os.path.join(json_folder, \'Resultado_analisis_combinados.json\')\n    with open(output_json_path, \'w\') as json_file:\n        json.dump(all_analysis_results, json_file,

In [10]:
def analyze_keypoints(video_path):
    keypoints_list = detect_keypoints(video_path)
    fps = None
    analysis_results = []

    for frame_idx, keypoints in enumerate(keypoints_list):
        second = frame_idx // fps if fps else 0

        if not keypoints:
            analysis_results.append({
                'segundo': second,
                'frame': frame_idx,
                'video_name': None,
                'angulo_hombro_izquierdo': None,
                'angulo_del_hombro_derecho': None,
                'angulo_codo_izquierdo': None,
                'angulo_codo_derecho': None,
                'angulo_de_muneca_izquierda': None,
                'angulo_de_muneca_derecha': None,
                'angulo_mano_izquierdo': None,
                'angulo_mano_derecho': None,
                'posicion_hombro_izquierdo': None,
                'posicion_hombro_derecho': None,
                'posicion_codo_izquierdo': None,
                'posicion_codo_derecho': None,
                'posicion_muneca_izquierda': None,
                'posicion_muneca_derecha': None,
                'posicion_mano_izquierda': None,
                'posicion_mano_derecha': None,
                'keypoints': []
            })
            continue

        # Extraer puntos clave izquierdos
        left_shoulder = keypoints[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
        left_elbow = keypoints[mp_pose.PoseLandmark.LEFT_ELBOW.value]
        left_wrist = keypoints[mp_pose.PoseLandmark.LEFT_WRIST.value]
        left_hand = keypoints[mp_pose.PoseLandmark.LEFT_INDEX.value]

        # Extraer puntos clave derechos
        right_shoulder = keypoints[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
        right_elbow = keypoints[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
        right_wrist = keypoints[mp_pose.PoseLandmark.RIGHT_WRIST.value]
        right_hand = keypoints[mp_pose.PoseLandmark.RIGHT_INDEX.value]

        # Calcular ángulos izquierdos
        left_shoulder_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
        left_elbow_angle = calculate_angle(left_elbow, left_wrist, left_hand)
        left_wrist_angle = calculate_angle(left_elbow, left_wrist, left_hand)
        left_hand_angle = calculate_angle(left_elbow, left_wrist, left_hand)

        # Calcular ángulos derechos
        right_shoulder_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
        right_elbow_angle = calculate_angle(right_elbow, right_wrist, right_hand)
        right_wrist_angle = calculate_angle(right_elbow, right_wrist, right_hand)
        right_hand_angle = calculate_angle(right_elbow, right_wrist, right_hand)

        # Si no se ha establecido aún el FPS, se establece ahora
        if fps is None:
            cap = cv2.VideoCapture(video_path)
            fps = int(cap.get(cv2.CAP_PROP_FPS))
            cap.release()

        analysis_results.append({
            'segundo': second,
            'frame': frame_idx,
            'analisis_video': video_path,
            'angulo_hombro_izquierdo': left_shoulder_angle,
            'angulo_del_hombro_derecho': right_shoulder_angle,
            'angulo_codo_izquierdo': left_elbow_angle,
            'angulo_codo_derecho': right_elbow_angle,
            'angulo_de_muneca_izquierda': left_wrist_angle,
            'angulo_de_muneca_derecha': right_wrist_angle,
            'angulo_mano_izquierdo': left_hand_angle,
            'angulo_mano_derecho': right_hand_angle,
            'posicion_hombro_izquierdo': left_shoulder,
            'posicion_hombro_derecho': right_shoulder,
            'posicion_codo_izquierdo': left_elbow,
            'posicion_codo_derecho': right_elbow,
            'posicion_muneca_izquierda': left_wrist,
            'posicion_muneca_derecha': right_wrist,
            'posicion_mano_izquierda': left_hand,
            'posicion_mano_derecha': right_hand,
            'keypoints': keypoints
        })

    return analysis_results, fps

In [11]:
def draw_keypoints_and_angles(video_path, output_folder, output_video_folder, json_folder):
    analysis_results, fps = analyze_keypoints(video_path)

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error al abrir el video: {video_path}")
        return
    
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    video_frame_folder = os.path.join(output_folder, video_name)
    output_video_name = f"{video_name}_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.mp4"
    output_video_path = os.path.join(output_video_folder, output_video_name)
    json_filename = f"{video_name}_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    json_path = os.path.join(json_folder, json_filename)

    # Crear carpetas si no existen
    if not os.path.exists(video_frame_folder):
        os.makedirs(video_frame_folder)
    if not os.path.exists(output_video_folder):
        os.makedirs(output_video_folder)
    if not os.path.exists(json_folder):
        os.makedirs(json_folder)

    # Crear el video marcado
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    frame_count = 0
    for i in range(len(analysis_results)):
        ret, frame = cap.read()
        if not ret:
            break

        keypoints = analysis_results[i]['keypoints']
        left_shoulder_angle = analysis_results[i]['angulo_hombro_izquierdo']
        right_shoulder_angle = analysis_results[i]['angulo_del_hombro_derecho']
        left_elbow_angle = analysis_results[i]['angulo_codo_izquierdo']
        right_elbow_angle = analysis_results[i]['angulo_codo_derecho']
        left_wrist_angle = analysis_results[i]['angulo_de_muneca_izquierda']
        right_wrist_angle = analysis_results[i]['angulo_de_muneca_derecha']
        left_hand_angle = analysis_results[i]['angulo_mano_izquierdo']
        right_hand_angle = analysis_results[i]['angulo_mano_derecho']

        # Dibujar puntos clave y conexiones con el color original
        if keypoints:
            for j, point in enumerate(keypoints):
                cv2.circle(frame, (int(point[0] * frame.shape[1]), int(point[1] * frame.shape[0])), 5, (0, 255, 0), -1)

                # Añadir texto con información relevante junto a cada punto
                if j == mp_pose.PoseLandmark.LEFT_SHOULDER.value:
                    cv2.putText(frame, f'Angulo del hombro izquierdo: {int(left_shoulder_angle) if left_shoulder_angle else 0}',
                                (int(point[0] * frame.shape[1]) + 10, int(point[1] * frame.shape[0]) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
                elif j == mp_pose.PoseLandmark.RIGHT_SHOULDER.value:
                    cv2.putText(frame, f'Angulo del hombro derecho: {int(right_shoulder_angle) if right_shoulder_angle else 0}',
                                (int(point[0] * frame.shape[1]) + 10, int(point[1] * frame.shape[0]) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
                elif j == mp_pose.PoseLandmark.LEFT_ELBOW.value:
                    cv2.putText(frame, f'Angulo del codo izquierdo: {int(left_elbow_angle) if left_elbow_angle else 0}',
                                (int(point[0] * frame.shape[1]) + 10, int(point[1] * frame.shape[0]) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
                elif j == mp_pose.PoseLandmark.RIGHT_ELBOW.value:
                    cv2.putText(frame, f'Angulo del codo derecho: {int(right_elbow_angle) if right_elbow_angle else 0}',
                                (int(point[0] * frame.shape[1]) + 10, int(point[1] * frame.shape[0]) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
                elif j == mp_pose.PoseLandmark.LEFT_WRIST.value:
                    cv2.putText(frame, f'Angulo de la muneca izquierda: {int(left_wrist_angle) if left_wrist_angle else 0}',
                                (int(point[0] * frame.shape[1]) + 10, int(point[1] * frame.shape[0]) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
                elif j == mp_pose.PoseLandmark.RIGHT_WRIST.value:
                    cv2.putText(frame, f'Angulo de la muneca derecha: {int(right_wrist_angle) if right_wrist_angle else 0}',
                                (int(point[0] * frame.shape[1]) + 10, int(point[1] * frame.shape[0]) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
                elif j == mp_pose.PoseLandmark.LEFT_INDEX.value:
                    cv2.putText(frame, f'Angulo de la mano izquierda: {int(left_hand_angle) if left_hand_angle else 0}',
                                (int(point[0] * frame.shape[1]) + 10, int(point[1] * frame.shape[0]) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
                elif j == mp_pose.PoseLandmark.RIGHT_INDEX.value:
                    cv2.putText(frame, f'Angulo de la mano derecha: {int(right_hand_angle) if right_hand_angle else 0}',
                                (int(point[0] * frame.shape[1]) + 10, int(point[1] * frame.shape[0]) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)

        # Guardar el frame en la carpeta de salida con el nombre del video seguido de su numeración
        output_frame_path = os.path.join(video_frame_folder, f'{video_name}_frame_{frame_count:04d}.png')
        cv2.imwrite(output_frame_path, frame)
        frame_count += 1

        # Guardar el frame en el video marcado
        out.write(frame)

    cap.release()
    out.release()

    # Guardar el archivo JSON con los resultados del análisis
    with open(json_path, 'w') as json_file:
        json.dump(analysis_results, json_file, indent=4)

    print(f"Frames exportados a: {video_frame_folder}")
    print(f"Video exportado a: {output_video_path}")
    print(f"JSON exportado a: {json_path}")
    return video_frame_folder, output_video_path, json_path


In [12]:
video_path = videos[0]
output_folder = 'C:\\Tesis\\TestErgo\\resultados'
output_video_folder = 'C:\\Tesis\\TestErgo\\videoMarcado'
json_folder = 'C:\\Tesis\\TestErgo\\archivos_json'

draw_keypoints_and_angles(video_path, output_folder, output_video_folder, json_folder)

c:\Tesis\TesisOCRA\TesisCheckListOCRA\python\venv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Frames exportados a: C:\Tesis\TestErgo\resultados\etiquetado
Video exportado a: C:\Tesis\TestErgo\videoMarcado\etiquetado_20240718_200217.mp4
JSON exportado a: C:\Tesis\TestErgo\archivos_json\etiquetado_20240718_200217.json


('C:\\Tesis\\TestErgo\\resultados\\etiquetado',
 'C:\\Tesis\\TestErgo\\videoMarcado\\etiquetado_20240718_200217.mp4',
 'C:\\Tesis\\TestErgo\\archivos_json\\etiquetado_20240718_200217.json')

Factor Recuperacion

In [13]:
mp_pose = mp.solutions.pose

def detectar_movimiento(frame, pose, umbral_movimiento=0.05):
    # Convertir la imagen de BGR a RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(frame_rgb)
    
    # Si no se detecta ninguna pose, considerar que no hay movimiento
    if not results.pose_landmarks:
        return False

    # Extraer los puntos clave de la pose detectada
    keypoints = results.pose_landmarks.landmark
    keypoints_array = np.array([[kp.x, kp.y, kp.z] for kp in keypoints])

    # Comparar con la pose anterior para detectar movimiento
    if not hasattr(detectar_movimiento, "pose_anterior"):
        detectar_movimiento.pose_anterior = keypoints_array
        return False

    # Calcular la distancia media entre los puntos clave actuales y los anteriores
    diferencia = np.linalg.norm(detectar_movimiento.pose_anterior - keypoints_array, axis=1)
    movimiento = np.mean(diferencia) > umbral_movimiento

    # Actualizar la pose anterior
    detectar_movimiento.pose_anterior = keypoints_array

    return movimiento

def analizar_pausas_y_recuperacion(video_path, umbral_inactividad=10):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error al abrir el video: {video_path}")
        return

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = frame_count / fps
    print(f"Duración del video: {duration} segundos")

    # Variables para almacenar las pausas y tiempos de inactividad
    pausas = []
    #inactividad = []

    # Variables temporales para detectar pausas
    inicio_pausa = None
    duracion_pausa = 0
    en_pausa = False

    with mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        for i in range(frame_count):
            ret, frame = cap.read()
            if not ret:
                break

            movimiento_detectado = detectar_movimiento(frame, pose)

            if not movimiento_detectado:
                if not en_pausa:
                    inicio_pausa = i / fps
                    en_pausa = True
                duracion_pausa += 1 / fps
            else:
                if en_pausa and duracion_pausa >= umbral_inactividad:
                    pausas.append((inicio_pausa, inicio_pausa + duracion_pausa))
                inicio_pausa = None
                duracion_pausa = 0
                en_pausa = False

        # Agregar la última pausa si terminó en el último frame
        if en_pausa and duracion_pausa >= umbral_inactividad:
            pausas.append((inicio_pausa, inicio_pausa + duracion_pausa))

    cap.release()

    # Imprimir resultados
    print("Pausas detectadas:")
    for inicio, fin in pausas:
        print(f"Inicio: {inicio:.2f} segundos, Fin: {fin:.2f} segundos, Duración: {fin - inicio:.2f} segundos")

    return pausas

# Ejemplo de uso:
video_path = videos[0]
pausas = analizar_pausas_y_recuperacion(video_path)

Duración del video: 189.08695652173913 segundos
Pausas detectadas:


Paso 2: Determinación de Tiempos de Recuperación

In [16]:
def calcular_factor_recuperacion(pauses, duracion_turno, tiempo_almuerzo, tiempo_no_repetitivo):
    # Calcular la duración total de las pausas en minutos
    total_duracion_pausas = sum((pausa[1] - pausa[0]) for pausa in pauses) / 60.0  # Convertir de segundos a minutos

    # Calcular el tiempo neto de trabajo repetitivo (TNTR)
    TNTR = duracion_turno - (total_duracion_pausas + tiempo_almuerzo + tiempo_no_repetitivo)

    # Calcular el Factor de Recuperación (FR)
    # Esto dependerá de la fórmula o tabla específica que utilices
    FR = total_duracion_pausas / TNTR  # Ejemplo simple, ajusta git según tu metodología

    return FR, TNTR

# Ejemplo de uso
pauses = [(600, 900), (1800, 2100)]  # Pausas en segundos
duracion_turno = 480  # Duración del turno en minutos
tiempo_almuerzo = 45  # Tiempo de almuerzo en minutos
tiempo_no_repetitivo = 60  # Tiempo de trabajo no repetitivo en minutos

FR, TNTR = calcular_factor_recuperacion(pauses, duracion_turno, tiempo_almuerzo, tiempo_no_repetitivo)
print(f"Factor de Recuperación (FR): {FR}")
print(f"Tiempo Neto de Trabajo Repetitivo (TNTR): {TNTR} minutos")

Factor de Recuperación (FR): 0.0273972602739726
Tiempo Neto de Trabajo Repetitivo (TNTR): 365.0 minutos


Análisis de actividad, pausas y cálculo del Factor de Recuperación (FR)

In [ ]:
# Función para identificar acciones técnicas dinámicas y estáticas
def classify_technical_actions(keypoints):
    atd = 0
    ate = 0
    dynamic_actions = 0
    static_actions = 0

    for i in range(1, len(keypoints)):
        movement = keypoints[i] - keypoints[i - 1]
        if movement.any():
            dynamic_actions += 1
        else:
            static_actions += 1

    # Calcular ATD según la tabla proporcionada
    actions_per_minute = (dynamic_actions / (len(keypoints) / 30)) * 60
    if actions_per_minute <= 20:
        atd = 0
    elif actions_per_minute <= 30:
        atd = 1
    elif actions_per_minute <= 40:
        atd = 3
    elif actions_per_minute <= 50:
        atd = 4
    elif actions_per_minute <= 60:
        atd = 6
    elif actions_per_minute <= 70:
        atd = 8
    else:
        atd = 10

    # Calcular ATE según la tabla proporcionada
    if static_actions >= (2 / 3) * len(keypoints):
        ate = 2.5
    elif static_actions >= (3 / 3) * len(keypoints):
        ate = 4.5

    return atd, ate

# Función para analizar la actividad y pausas en el video
def analyze_activity_and_breaks(analysis_results, break_threshold=5):
    activity_periods = []
    break_periods = []

    current_activity_start = None
    current_break_start = None
    in_activity = False

    for second, keypoints in enumerate(analysis_results):
        if keypoints:  # Si hay puntos clave detectados, se considera actividad
            if not in_activity:
                current_activity_start = second
                if current_break_start is not None:
                    break_duration = second - current_break_start
                    if break_duration >= break_threshold:
                        break_periods.append((current_break_start, second))
                in_activity = True
        else:  # Si no hay puntos clave detectados, se considera pausa
            if in_activity:
                activity_duration = second - current_activity_start
                activity_periods.append((current_activity_start, second))
                current_break_start = second
                in_activity = False

    # Agregar el último periodo de actividad o pausa
    if in_activity and current_activity_start is not None:
        activity_periods.append((current_activity_start, analysis_results[-1]))
    if not in_activity and current_break_start is not None:
        break_periods.append((current_break_start, analysis_results[-1]))

    return activity_periods, break_periods

# Análisis de actividad, pausas y cálculo del Factor de Frecuencia (FF)
for video_path in videos:
    analysis_results = analyze_keypoints(video_path)
    activity_periods, break_periods = analyze_activity_and_breaks(analysis_results)

    # Calcular el total de actividad y pausas
    total_activity = sum(end - start for start, end in activity_periods)
    total_break = sum(end - start for start, end in break_periods)
    total_duration = total_activity + total_break

    # Clasificar las acciones técnicas y calcular ATD y ATE
    atd, ate = classify_technical_actions(analysis_results)

    # Calcular el Factor de Frecuencia (FF)
    ff = max(atd, ate)

    # Imprimir los resultados
    print(f"Video: {os.path.basename(video_path)}")
    print("Periodos de Actividad:", activity_periods)
    print("Periodos de Pausas:", break_periods)
    print(f"Total Actividad: {total_activity} segundos")
    print(f"Total Pausas: {total_break} segundos")
    print(f"Acciones Técnicas Dinámicas (ATD): {atd}")
    print(f"Acciones Técnicas Estáticas (ATE): {ate}")
    print(f"Factor de Frecuencia (FF): {ff}")

Factor fuerza

In [18]:
import analyze_keypoints

# Función para clasificar la fuerza aplicada en las acciones técnicas
def classify_force_actions(technical_actions):
    force_scores = []
    for action in technical_actions:
        # Aquí debes agregar tu lógica para evaluar la fuerza aplicada
        # Por ahora, asumiremos fuerza moderada para todos los movimientos
        force_scores.append(1)  # Cambia esto según tu evaluación de fuerza

    # Calcular la puntuación de la fuerza (FFu) basada en la evaluación
    if force_scores.count(4) > 0:
        ffu = 4
    elif force_scores.count(3) > 0:
        ffu = 3
    elif force_scores.count(2) > 0:
        ffu = 2
    elif force_scores.count(1) > 0:
        ffu = 1
    else:
        ffu = 0

    return ffu

# Función para clasificar las acciones técnicas dinámicas y estáticas
def classify_technical_actions(keypoints_list):
    dynamic_actions = 0
    static_actions = 0

    for i in range(1, len(keypoints_list)):
        if keypoints_list[i] is not None and keypoints_list[i - 1] is not None:
            movement = np.linalg.norm(keypoints_list[i] - keypoints_list[i - 1])
            if movement > 0:
                dynamic_actions += 1
            else:
                static_actions += 1

    # Calcular ATD según la tabla proporcionada
    actions_per_minute = (dynamic_actions / (len(keypoints_list) / 30)) * 60
    if actions_per_minute <= 20:
        atd = 0
    elif actions_per_minute <= 30:
        atd = 1
    elif actions_per_minute <= 40:
        atd = 3
    elif actions_per_minute <= 50:
        atd = 4
    elif actions_per_minute <= 60:
        atd = 6
    elif actions_per_minute <= 70:
        atd = 8
    else:
        atd = 10

    # Calcular ATE según la tabla proporcionada
    if static_actions >= (2 / 3) * len(keypoints_list):
        ate = 2.5
    elif static_actions >= (3 / 3) * len(keypoints_list):
        ate = 4.5
    else:
        ate = 0

    return atd, ate

# Análisis de actividad, pausas y cálculo del Factor de Frecuencia (FF)
def analyze_activity_and_calculate_factors(video_path):
    keypoints_list = analyze_video(video_path)

    # Clasificar las acciones técnicas y calcular ATD y ATE
    atd, ate = classify_technical_actions(keypoints_list)

    # Calcular el Factor de Frecuencia (FF)
    ff = max(atd, ate)

    # Clasificar y calcular el Factor de Fuerza (FFu)
    ffu = classify_force_actions(keypoints_list)

    # Imprimir los resultados
    print(f"Video: {os.path.basename(video_path)}")
    print(f"Acciones Técnicas Dinámicas (ATD): {atd}")
    print(f"Acciones Técnicas Estáticas (ATE): {ate}")
    print(f"Factor de Frecuencia (FF): {ff}")
    print(f"Factor de Fuerza (FFu): {ffu}")

# Ruta del video
video_path = "ruta_al_video.mp4"

# Ejecutar el análisis y calcular los factores
analyze_activity_and_calculate_factors(video_path)

NameError: name 'analyze_video' is not defined